# <center> AI4LR - Final Assignment [100 Points]

In [1]:
# IMPORTANT
Your_name = "Pranav Raj H"
Your_emailid = "vinstarpranav@gmail.com"

---

## Dataset
- Add [this dataset](https://www.kaggle.com/datasets/romainpessia/artificial-lunar-rocky-landscape-dataset) to your Kaggle notebook.

## Instructions for this project
- Don't forget to turn on GPU for training your model(s).
- Without changing anything in the notebook if you run it then you will get the val_iou_score of around 0.20.
- Your goal of this project is to increase this val_iou_score as much as you can.
- Evaluation of this project will be based on your best acquired val_iou_score seen in the notebook.
- Your val_iou_score will be the percentage you will receive for this project. 
- If your best val_iou_score is 0.41 then you will score 41/100 points in this project.
- Try to avoid any errors before submitting your notebook.

## Tips to increase the performance of your model
- Increase the number of epochs.
- Increase the number of layers in your model.
- Using SOTA high performance networks with transfer learning.
- Using callbacks and carefully observing your model performance.
- You can use the methods taught to you in this training program or any other methods of your own choice to increase the performance!

## Guidelines on making changes to this notebook
**1)** Add a descriptive comment to your code for whatever changes you are making in this notebook.
- For example, if you are adding an extra Conv2D layer, write about all the aspects of the Conv2D layer you are adding.
- The commnt should be placed at the point where the layer will be added.

**2)** Show model properties before and after the changes were made.
- For example, if you changed the layers - added, deleted, e.t.c.

**3)** If you use new data preprocessing techniques that are not already part of this notebook, you must explain their inner workings using markdown cells.
- Without this explanation, your techniques will not be considered for evaluation.
- Use texts and images to explain this process.

**4)** Make use of tables and plots that contributed to the improvement of your model.
- Assume if increasing the epochs and decreasing the learning rate contributed in the improvement of your model.
- You will first show these improvements using plots of val_iou_scores vs epochs as well as val_iou_scores vs learning rate.0
- Then make use of tables to show iou scores for different learning rates.
- For example, table 1 for lr_1 to show iou values for epochs 30 t0 50, table 2 to show iou values from epochs 30 to 50, and so on.
- It is therefore advised to work on one improvement, optimize it, plot it, document it, then proceed to the next improvement - till you get a satisfactory IOU score.

**5)** Final improvement summary table.
- Prepare a table with columns (changes, improvments description, increase in iou from, increase in iou to)
- List out all the changes you made to improve your final model performance.

## Coding for this project

In [2]:
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00


In [3]:
# import the necessary Library

import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras 
from sklearn.model_selection import train_test_split

Segmentation Models: using `keras` framework.


* Provide environment variable SM_FRAMEWORK=keras / SM_FRAMEWORK=tf.keras before import segmentation_models
* Change framework sm.set_framework('keras') / sm.set_framework('tf.keras')

In [4]:
# Setting framework environment
os.environ["SM_FRAMEWORK"] = "tf.keras"
sm.set_framework('tf.keras')
keras.backend.set_image_data_format('channels_last')

## Data Preprocessing Pipeline

In [5]:
H = 256 # height of image
W = 256 # width of image

'''This function is used to return the list of path for images and masks in
sorted order from the given directory respectively.'''
# function to return list of image paths and mask paths 
def process_data(IMG_DIR, MASK_DIR):
    images = [os.path.join(IMG_DIR, x) for x in sorted(os.listdir(IMG_DIR))]
    masks = [os.path.join(MASK_DIR, x) for x in sorted(os.listdir(MASK_DIR))]

    return images, masks

'''This function is used to return splitted list of images and corresponding 
mask paths in train and test by providing test size.'''
# function to load data and train test split
def load_data(IMG_DIR, MASK_DIR):
    X, y = process_data(IMG_DIR, MASK_DIR)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test

'''This function is used to read images. It takes image path as input. 
After reading image it is resized by width and height provide above(256 x 256). 
Next normalization is done by dividing each values with 255. And the result is returned.'''
# function to read image
def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

'''This function is used to read masks.'''
# function to read mask
def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x.astype(np.int32)
    return x

'''This function is used to generate tensorflow data pipeline. 
The tensorflow data pipeline is mapped to function ‘preprocess’ .'''
# function for tensorflow dataset pipeline
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)
    return dataset

'''This function takes image and mask path. 
It reads the image and mask as provided by paths. 
Mask is one hot encoded for multi class segmentation (here 4 class).'''
# function to read image and mask amd create one hot encoding for mask
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image = read_image(x)
        mask = read_mask(y)

        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, 4, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 4])

    return image, mask

## Load the dataset

In [6]:
'''RENDER_IMAGE_DIR_PATH: ‘Path of image directory’
GROUND_MASK_DIR_PATH: ‘Path of mask directory’

Here load_data function is called. This will load the dataset paths and 
split it into X_train, X_test, y_train, y_test '''

RENDER_IMAGE_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/render'
GROUND_MASK_DIR_PATH = '../input/artificial-lunar-rocky-landscape-dataset/images/ground'

X_train, X_test, y_train, y_test = load_data(RENDER_IMAGE_DIR_PATH, GROUND_MASK_DIR_PATH)
print(f"Dataset:\n Train: {len(X_train)} \n Test: {len(X_test)}")

Dataset:
 Train: 7812 
 Test: 1954


## Generate tensorflow data pipeline

In [7]:
batch_size = 8

'''Here the tf_dataset function is called will generate the tensorflow data pipeline.'''
# calling tf_dataset
train_dataset = tf_dataset(X_train, y_train, batch=batch_size)
valid_dataset = tf_dataset(X_test, y_test, batch=batch_size)

2023-01-27 13:42:01.408252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 13:42:01.409249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 13:42:01.531825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 13:42:01.532766: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-27 13:42:01.533668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

## Creating U-net Architecture

In [8]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

'''conv_block it is used to create one block with two convolution layer 
followed by BatchNormalization and activation function relu. 
If the pooling is required then Maxpool2D is applied and return it else not.'''
# function to create convolution block
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x

'''build_unet it is used to create the U-net architecture.'''
# function to build U-net
def build_unet(shape, num_classes):
    inputs = Input(shape)

    """ Encoder """
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """ Bridge """
    b1 = conv_block(p4, 128, pool=False)

    """ Decoder """
    u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """ Output layer """
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)

In [12]:
# calling build_unet function
model = build_unet((256, 256, 3), 4)

#printing model summary
# model.summary()

## Load model and compile

In [10]:
# importing libraries
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from segmentation_models.metrics import iou_score
import datetime, os

""" Defining Hyperparameters """
img_shape = (256, 256, 3)
num_classes = 4
lr = 1e-4
batch_size = 16
epochs = 5

""" Model building and compiling """
model = build_unet(img_shape, num_classes)
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(lr), 
              metrics=[iou_score])


train_steps = len(X_train)//batch_size
valid_steps = len(X_test)//batch_size

## Train model

In [11]:
'''model.fit is used to train the model'''
model_history = model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
    )

2023-01-27 13:42:06.024871: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2023-01-27 13:42:09.860773: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


488/488 [==============================] - 163s 309ms/step - loss: 0.3834 - iou_score: 0.1429 - val_loss: 0.2605 - val_iou_score: 0.1608
Epoch 2/5
488/488 [==============================] - 144s 296ms/step - loss: 0.2127 - iou_score: 0.1719 - val_loss: 0.1928 - val_iou_score: 0.1764
Epoch 3/5
488/488 [==============================] - 90s 181ms/step - loss: 0.1493 - iou_score: 0.1855 - val_loss: 0.1224 - val_iou_score: 0.1916
Epoch 4/5
488/488 [==============================] - 86s 177ms/step - loss: 0.1096 - iou_score: 0.1934 - val_loss: 0.0937 - val_iou_score: 0.1956
Epoch 5/5
488/488 [==============================] - 86s 176ms/step - loss: 0.0834 - iou_score: 0.1983 - val_loss: 0.0601 - val_iou_score: 0.1877


In [15]:
# importing packages to visualize
import seaborn
from matplotlib import pyplot

## Try 1: Running with no changes

In [18]:
model_history.

AttributeError: 'History' object has no attribute 'summary'

---
# <center> THE END